


<h2 align="center">
    Project: Advanced SQL Queries - #3
</h2>




**Project Description:**

To work with the New York City Airbnb dataset in this project, we need to create and connect to an SQLite database as seen in the steps within this notebook.



**Dataset**
- You can find the New York City Airbnb dataset on Kaggle: [New York City Airbnb Dataset](https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data)


## Introduction
This SQL project  uses a real-world dataset. In this project, we will work with the New York City Airbnb dataset and performadvance data analysis tasks.

## Table of Contents
1. [Libraries and Database Setup](#database-setup)
2. [Loading the Dataset](#loading-dataset)
3. [Advanced SQL Queries](#sql-queries)
4. [Closing Connection](#closing-connection)
5. [Summary of Advanced SQL Commands](#summary-commands)

# Section 1: Libraries and Database Setup <a name="database-setup"></a>

###  1.1: Import Libraries
Let's start by importing the necessary libraries.


In [1]:
!pip install kaggle
! mkdir ~/.kaggle
!pip install -q kaggle


In [2]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download -d dgomonov/new-york-city-airbnb-open-data

Dataset URL: https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data
License(s): CC0-1.0
 82% 2.00M/2.44M [00:00<00:00, 2.97MB/s]
100% 2.44M/2.44M [00:00<00:00, 3.07MB/s]


In [4]:
!unzip /content/new-york-city-airbnb-open-data.zip

Archive:  /content/new-york-city-airbnb-open-data.zip
  inflating: AB_NYC_2019.csv         
  inflating: New_York_City_.png      


In [5]:
import pandas as pd
import numpy as np
import sqlite3

###  1.2: Connect to the Database

Next, establish a connection to the SQLite database named 'airbnb.db'.
Use the sqlite3 library to create a connection object and store it in a variable called 'conn'.


In [6]:


# Connect to the SQLite database
conn = sqlite3.connect('airbnb.db')
cursor = conn.cursor()

# Section 2: Loading the Dataset <a name="loading-dataset"></a>

###  2.1: Load the Dataset
- We will work with the 'listings' table from the New York City Airbnb dataset.
- Load the dataset from the CSV file 'AB_NYC_2019.csv' into a DataFrame named 'df'.
- Then, import the DataFrame into the 'listings' table in the database using the to_sql() method.

\\




In [7]:


# Load the dataset into the SQLite database
data_path = '/content/AB_NYC_2019.csv'
df = pd.read_csv(data_path)
df.to_sql('listings', conn, if_exists='replace', index=False)

48895

# Section 3: Advanced SQL Tasks <a name="sql-queries"></a>

Query the database: Write SQL queries to analyze the data in the SQLite database and can use the sqlite3 library to execute the queries and fetch the results.

###  3.1: Analyzing Host Performance
- Find the top 10 hosts with the highest average ratings.
- Include host's name, number of reviews, and average rating.

In [8]:


query = """
-- 1. Calculate the number of reviews received by each host and name the result as 'num_reviews'

SELECT host_name, COUNT(*) AS num_reviews, AVG(CAST(number_of_reviews AS FLOAT)) AS avg_rating FROM listings
-- 2. Calculate the average rating (reviews per month) for each host and name the result as 'avg_rating'
-- 3. Group the results by host name
Group By host_name
-- 4. Order the results by average rating in descending order
ORDER BY avg_rating DESC
-- 5. Display the top 10 hosts with their names, the number of reviews, and average rating
LIMIT 10;
"""

In [9]:


result = pd.read_sql_query(query, conn)
print("\nAnalyzing Host Performance:")
print(result)


Analyzing Host Performance:
        host_name  num_reviews  avg_rating
0            Dona            2       602.5
1             Asa            1       488.0
2  Dennis & Naoko            1       441.0
3         Miss Dy            1       434.0
4        Shunichi            1       430.0
5    Gladys & Bob            1       414.0
6             Sne            1       396.0
7          Malini            1       392.0
8            Amia            1       385.0
9            J. E            1       378.0


###  3.2: Analyzing Monthly Price Trends
- Display the month, average price, and number of listings for each month.

In [10]:


query = """
-- 1. Use the `strftime('%m', last_review)` function to extract the month from the 'last_review' column and name it as 'month'
-- 2. Calculate the average price for listings in each month and name the result as 'avg_price'
SELECT strftime('%m', last_review) AS month, AVG(CAST(price AS FLOAT)) AS avg_price, COUNT(*) AS num_listings FROM listings
-- 3. Group the results by the 'month' column
-- 3. Calculate the number of listings in each month and name the result as 'num_listings'

-- 4. Group the results by the 'month' column
Group By month
-- 5. Order the results by average price in descending order
ORDER BY avg_price DESC;
"""

In [11]:


result = pd.read_sql_query(query, conn)
print("Analyzing Monthly Price Trends:")
print(result)

Analyzing Monthly Price Trends:
   month   avg_price  num_listings
0   None  192.919021         10052
1     12  160.796610          1770
2     02  155.716883           770
3     01  150.478904          2773
4     09  149.494463          1535
5     07  142.290382          5937
6     10  142.065330          1546
7     06  140.769667         13589
8     04  139.504979          2109
9     11  138.532815          1158
10    05  137.909594          4701
11    03  136.836957          1288
12    08  132.133173          1667


###  3.3: Find the top 5 neighborhoods with the highest price variability
-  Find the top 5 neighborhoods with the highest price variability.
-  Display neighborhood and price variability.


In [12]:


query = """
-- 1. Create a Common Table Expression (CTE) named 'price_variability' that calculates the following for each neighborhood:
--    - Calculate the average price and name it as 'avg_price'
--    - Calculate the maximum price and name it as 'max_price'
--    - Calculate the minimum price and name it as 'min_price'
WITH price_variability AS (
    SELECT neighbourhood, AVG(CAST(price AS FLOAT)) AS avg_price, MAX(CAST(price AS FLOAT)) AS max_price, MIN(CAST(price AS FLOAT)) AS min_price
    FROM listings
    GROUP BY neighbourhood
)
-- 2. In the main query, select the 'neighbourhood' column and calculate the price variability as the difference between 'max_price' and 'min_price'
SELECT neighbourhood, max_price - min_price AS price_variability FROM price_variability
-- 3. Order the results by price variability in descending order
ORDER BY price_variability DESC
-- 4. Limit the results to the top 5 neighborhoods
LIMIT 5;
"""

In [13]:


result = pd.read_sql_query(query, conn)
print("Advanced Aggregation:")
print(result)

Advanced Aggregation:
     neighbourhood  price_variability
0       Greenpoint            10000.0
1  Upper West Side             9990.0
2          Astoria             9975.0
3  Lower East Side             9970.0
4      East Harlem             9969.0


###  3.4: Advanced Data Manipulation
- Calculate days since the last review and identify top 10 listings.
- Display listing ID, name, and days since the last review.

In [14]:


query = """
-- 1. Select the following columns from the 'listings' table:
--    - 'id'
--    - 'name'
--    - Calculate the number of days since the last review and name it as 'days_since_last_review.'
--      Use the 'julianday' function to calculate the difference in days between the current date and the 'last_review' date
SELECT id, name, julianday('now') - julianday(last_review) AS days_since_last_review FROM listings
-- 2. Order the results by 'days_since_last_review' in descending order
ORDER BY days_since_last_review DESC
-- 3. Limit the results to the top 10 listings
LIMIT 10;
"""

In [15]:


result = pd.read_sql_query(query, conn)
print("\nAdvanced Data Manipulation:")
print(result)


Advanced Data Manipulation:
       id                                               name  \
0   74860  Sunlit and Cozy Williamsburg/Greenpoint, Brooklyn   
1   40039                 Luxurious Condo in DUBMO with View   
2   81739                  Loft w/ Terrace @ Box House Hotel   
3   28396                  Modern Apt with Spectacular Views   
4   32363                 Fully Furnished Basement Apartment   
5   27883                             East Village Sanctuary   
6    7801                   Sweet and Spacious Brooklyn Loft   
7  229874                     Oversized Studio in Park Slope   
8   98330                LOVELY APARTMENT IN THE HEART OF NY   
9  464231                     Large Room w/ Private Entrance   

   days_since_last_review  
0             4875.792212  
1             4847.792212  
2             4830.792212  
3             4701.792212  
4             4700.792212  
5             4618.792212  
6             4600.792212  
7             4595.792212  
8             

###  3.5:  Identifying Top Hosts by Revenue
- Calculate the total revenue generated by each host and rank them based on revenue.
- Display the host name, total revenue, and their rank.

In [16]:


query = """
-- 1. Select the following columns from the 'listings' table:
--    - 'host_name'
--    - Calculate the total revenue for each host by multiplying 'price' by 'minimum_nights' and name it as 'total_revenue.'
--    - Use the RANK() window function to determine the rank of each host by total revenue in descending order and name it as 'revenue_rank.'
SELECT host_name, SUM(CAST(price AS FLOAT) * minimum_nights) AS total_revenue, RANK() OVER (ORDER BY SUM(CAST(price AS FLOAT) * minimum_nights) DESC) AS revenue_rank FROM listings
-- 2. Group the results by 'host_name.'
GROUP BY host_name
-- 3. Order the results by 'total_revenue' in descending order.
ORDER BY total_revenue DESC
-- 4. Limit the results to the top 10 hosts.
LIMIT 10;
"""

In [17]:


result = pd.read_sql_query(query, conn)
print("Identifying Top Hosts by Revenue:")
print(result)

Identifying Top Hosts by Revenue:
      host_name  total_revenue  revenue_rank
0    Blueground      2258580.0             1
1         Jenny      1215111.0             2
2          Kara      1172637.0             3
3           Amy      1153867.0             4
4      Kathrine      1002082.0             5
5  Sonder (NYC)       950453.0             6
6         Iveta       857750.0             7
7        Noelle       732930.0             8
8       Pranjal       664320.0             9
9       Michael       610539.0            10


# Section 4: Closing the Database Connection <a name="closing-connection"></a>

###  4.1: Close the Cursor and Database Connection
- It's good practice to close the cursor and the database connection when you're done working with the database to free up system resources and maintain proper connection management.


In [18]:


# Make sure to include these lines of code at the end of your script to properly close the cursor and database connection.
cursor.close()
conn.close()

# Section 5: Summary of Advanced SQL Commands <a name="summary-commands"></a>

 Below is a summary of the advanced SQL commands used:

** 1.2: Connect to the Database**
- Established a connection to the SQLite database.

** 2.1: Load the Dataset**
- Loaded the dataset into a DataFrame.
- Imported the DataFrame into the database.

** 3.1: Identifying Top Hosts by Reviews and Ratings**
- SQL Commands: SELECT, COUNT(), AVG(), GROUP BY, ORDER BY, LIMIT
- Description: Calculate the number of reviews and average ratings for each host and identify the top hosts by ratings and reviews.

** 3.2: Analyzing Seasonal Price Trends**
- SQL Commands: SELECT, strftime(), AVG(), COUNT(), GROUP BY, ORDER BY
- Description: Analyze seasonal price trends by extracting the month from the last review date and calculating average prices.

** 3.3: Finding Neighborhoods with Price Variability**
- SQL Commands: WITH, SELECT, AVG(), MAX(), MIN(), GROUP BY, ORDER BY
- Description: Identify neighborhoods with the highest price variability by calculating the difference between the maximum and minimum prices.

** 3.4: Advanced Data Manipulation**
- SQL Commands: SELECT, julianday(), ORDER BY, LIMIT
- Description: Calculate the number of days since the last review for each listing and sort them by the days since the last review.

** 3.5: Identifying Top Hosts by Revenue**
- SQL Commands: SELECT, SUM(), RANK(), GROUP BY, ORDER BY, LIMIT
- Description: Determine the top hosts by total revenue generated from their listings, including a ranking based on revenue.

** 4.1: Close the Cursor and Database Connection**
- Closed the cursor and the database connection.

This summary provides an overview of the advanced SQL commands used in this project for data analysis tasks.